In [1]:
!rm -rf ../data/coco_eval/dt_dataset

In [2]:
gt_dataset_id = 88850
dt_dataset_id = 89542
save_path = '../data/coco_eval'

In [3]:
import supervisely as sly
import json
import os

api = sly.Api()

gt_project_id = api.dataset.get_info_by_id(gt_dataset_id).project_id
dt_project_id = api.dataset.get_info_by_id(dt_dataset_id).project_id
dataset_name = api.dataset.get_info_by_id(gt_dataset_id).name

# Download project
# sly.download(api, gt_project_id, save_path+"/gt_dataset", dataset_ids=[gt_dataset_id], log_progress=True)
sly.download(api, dt_project_id, save_path+"/dt_dataset", dataset_ids=[dt_dataset_id], log_progress=True)
gt_project = sly.read_project(save_path+"/gt_dataset")
dt_project = sly.read_project(save_path+"/dt_dataset")

# Get datasets
gt_dataset : sly.Dataset = next(iter(gt_project.datasets))
dt_dataset : sly.Dataset = next(iter(dt_project.datasets))
assert sorted(gt_dataset.get_items_names()) == sorted(dt_dataset.get_items_names())

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val'", "current": 0, "total": 200, "timestamp": "2024-04-16T06:31:27.298Z", "level": "info"}
{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val'", "current": 50, "total": 200, "timestamp": "2024-04-16T06:31:36.784Z", "level": "info"}
{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val'", "current": 100, "total": 200, "timestamp": "2024-04-16T06:31:45.885Z", "level": "info"}
{"message": "progress", "event_type": "EventType.PROGRESS", "subtask": "Downloading dataset: 'val'", "current": 150, "total": 200, "timestamp": "2024-04-16T06:31:55.068Z", "level": "info"}
{"m

In [4]:
def create_category_id_map(meta: sly.ProjectMeta):
    category_id_map = {}
    for i, obj_class in enumerate(meta.obj_classes):
        category_id_map[obj_class.name] = i + 1
    return category_id_map

def dataset_to_coco(dataset: sly.Dataset, category_id_map, is_prediction, meta):
    coco_data = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    annotation_id = 1
    image_id_map = {name: i for i, name in enumerate(sorted(dataset.get_items_names()))}

    # Add category information to COCO data
    for category_name, category_id in category_id_map.items():
        coco_data["categories"].append({"id": category_id, "name": category_name})

    # Processing annotations
    for name in sorted(dataset.get_items_names()):
        ann : sly.Annotation = dataset.get_ann(name, meta)
        image_id = image_id_map[name]
        coco_data["images"].append({
            "id": image_id,
            "file_name": dataset.get_img_path(name),
            "width": ann.img_size[1],
            "height": ann.img_size[0]
        })
        for label in ann.labels:
            if label.obj_class.geometry_type == sly.Rectangle:
                category_id = category_id_map[label.obj_class.name]
                bbox : sly.Rectangle = label.geometry.to_bbox()
                annotation = {
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": category_id,
                    "bbox": [bbox.left, bbox.top, bbox.width, bbox.height],
                    "area": bbox.width * bbox.height,
                    "iscrowd": 0
                }
                
                if is_prediction:
                    # Extract confidence score from the tag
                    score = label.tags.get("confidence").value
                    annotation["score"] = float(score)
                
                coco_data["annotations"].append(annotation)
                annotation_id += 1

    return coco_data

# Meta
meta = gt_project.meta

# Create a consistent category ID mapping for both datasets
category_id_map = create_category_id_map(meta)

# Convert datasets to COCO format with consistent category IDs
cocoGt = dataset_to_coco(gt_dataset, category_id_map, False, meta)
cocoDt = dataset_to_coco(dt_dataset, category_id_map, True, meta)

# assert image ids are the same
assert len(cocoGt["images"]) == len(cocoDt["images"])
for i in range(len(cocoGt["images"])):
    assert os.path.basename(cocoGt["images"][i]["file_name"]) == os.path.basename(cocoDt["images"][i]["file_name"])
    assert cocoGt["images"][i]["id"] == cocoDt["images"][i]["id"]

# Optionally save to JSON
with open('cocoGt.json', 'w') as f:
    json.dump(cocoGt, f)
with open('cocoDt.json', 'w') as f:
    json.dump(cocoDt['annotations'], f)

print("Conversion to COCO format completed and files saved.")

Conversion to COCO format completed and files saved.


In [5]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Load ground truth data
cocoGt=COCO("cocoGt.json")

# Load prediction data
cocoDt=cocoGt.loadRes("cocoDt.json")

# Initialize COCOeval object
cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')

# Evaluate on a subset of images (optional)
# cocoEval.params.imgIds = [5]  # Remove this line to evaluate on all images

# Run evaluation
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.227
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.213
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.047
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.268
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.225
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.254
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [ ]:
rtdetr-50 ep200: 0.177
rtdetr-50 ep60: 0.213
rtdetr-50 EMA ep60: 0.189
rtdetr-50 MODEL ep60: 0.188